In [1]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,f1_score

In [2]:
pd.options.display.float_format = '{:.20f}'.format

In [3]:
f = open("CriteoSearchData", 'r')
lines = f.readlines()

In [4]:
total_data = []
for i in range(0,1500000):
    total_data.append( lines[i].split('\t') )
    total_data[-1][-1] = total_data[-1][-1][:-2]

In [5]:
total_data = pd.DataFrame(total_data)

In [6]:
total_data.columns=['Sale' , 'SalesAmountInEuro' ,'Time_delay_for_conversion' ,'click_timestamp','nb_clicks_1week','product_price','product_age_group','device_type','audience_id','product_gender','product_brand','product_category_1','product_category_2','product_category_3','product_category_4','product_category_5','product_category_6','product_category_7','product_country','product_id','product_title','partner_id','user_id']

In [7]:
total_data['Sale'] = pd.to_numeric(total_data['Sale'])
total_data['SalesAmountInEuro'] = pd.to_numeric(total_data['SalesAmountInEuro'])
total_data['Time_delay_for_conversion'] = pd.to_numeric(total_data['Time_delay_for_conversion'])
total_data['click_timestamp'] = pd.to_numeric(total_data['click_timestamp'])
total_data['nb_clicks_1week'] = pd.to_numeric(total_data['nb_clicks_1week'])
total_data['product_price'] = pd.to_numeric(total_data['product_price'])

In [8]:
category_feature = [ col for col in total_data.columns if total_data[col].dtypes == "object"]

In [9]:
le = LabelEncoder()

In [10]:
for i in category_feature:
    le.fit(total_data[i])
    label = le.transform(total_data[i])
    total_data[i] = label

In [11]:
train_date = total_data[0:1000000]
test_date = total_data[1000000:1500000]

In [12]:
x_train = train_date[['product_price','product_age_group','device_type','nb_clicks_1week','product_gender','product_brand','product_category_1','product_country','product_title']]

In [13]:
x_test = test_date[['product_price','product_age_group','device_type','nb_clicks_1week','product_gender','product_brand','product_category_1','product_country','product_title']]

In [14]:
y_train = train_date['Sale']

In [15]:
y_test = test_date['Sale']

In [16]:
lr = LogisticRegression()

In [17]:
test1= x_train[['product_price','product_age_group','device_type','nb_clicks_1week','product_gender','product_brand','product_category_1','product_country','product_title']]

In [18]:
lr.fit(x_train,y_train)

C:\Users\AD0486\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [19]:
def result_def(x,y,lr):
    predict = lr.predict(x)
    score = f1_score(y,predict)
    per = lr.predict_proba(x)[:,1]
    line = []
    for i in range(1,len(x)+1):
        line.append(int(i))
    result = pd.concat([pd.DataFrame(line),pd.DataFrame(list(y)),pd.DataFrame(per)],axis=1)
    result.columns = ['Line Number','Target Label','Predicted Probability']
    return result, score

In [20]:
train_result,train_score = result_def(x_train,y_train,lr)

In [21]:
test_result,test_score = result_def(x_test,y_test,lr)

In [22]:
train_result.to_csv('train_result.txt', sep = '\t', index = False)

In [23]:
test_result.to_csv('test_result.txt', sep = '\t', index = False)

In [24]:
outfile = open("./line_venv/line_source/model.dat","wb")
pickle.dump(lr,outfile)
outfile.close()

In [25]:
outfile = open("model.dat","wb")
pickle.dump(lr,outfile)
outfile.close()

In [26]:
train_score

0.9608538237501594

In [27]:
test_score

0.9626631735850761